In [1]:
import numpy as np
import random
import copy
import pandas as pd
import heapq
import os
import gc
import time
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

PATH = os.path.abspath('.') 

In [56]:
# 一个lot的lotSplitingVector

class singleLotSplitingVec:
    
    """
    类含义：          一个lot的分批情况
    
    成员变量：
    self.lotSize      本批共有多少个工件
    self.sublotNum    子批数
    self.sublotSizes  list，每个子批包含多少个工件
    """
    
    def __init__(self, lotSize):
        """
        输入：
        lotSize     本批共有多少个工件
        """
        self.lotSize = lotSize

        
    def initializeLotSplitingVec(self):
        """
        功能：     随机初始化一个批的分批方案向量
        
        注意：     创建对象后需要使用该函数初始化后才有self.sublotNum和self.sublotSizes成员变量
        """
        # 随机生成子批数量，可限制子批数量，也可以不限制子批数量
#         self.sublotNum = random.randint(1, int(np.log2(self.lotSize)))  
#         self.sublotNum = random.randint(1, 2 * int(np.sqrt(self.lotSize)))  
#         self.lotNum = random.randint(1, self.lotSize)  # 
        if(self.lotSize > 30):
            self.sublotNum = random.randint(1, 2 * int(np.sqrt(30)))  # 超30的就不分那么多sublot了
        else:
            self.sublotNum = random.randint(1, 2 * int(np.sqrt(self.lotSize)))  
         
        # 随机生成每一个子批的批量
        self.sublotSizes = [1] * self.sublotNum
        cnt = self.sublotNum
        while(cnt < self.lotSize):
            self.sublotSizes[random.randint(1, self.sublotNum) - 1] += 1
            cnt += 1
    
    
    def mutateTwoSublot(self):
        """
        功能：      lot内变异两个sublot的size，即变异一个分批方案向量
        随机选择两个子批，重新随机生成这两个子批的批量，使之与变异前不同
        """
        if(self.sublotNum > 1):   # 当子批数大于1的时候，才可以发生变异  
            # 随机选择两个子批
            pos1 = random.randint(1, self.sublotNum) - 1
            pos2 = random.randint(1, self.sublotNum) - 1
            while(pos1 == pos2):
                pos2 = random.randint(1, self.sublotNum) - 1

            # 重新生成两个子批的批量数
            sumSize = self.sublotSizes[pos1] + self.sublotSizes[pos2]
            if(sumSize > 3):  # 如果选中的是两个1的sublot，或者是一个1一个2的sublot，则不进行变异，如果不是这种情况，才有以下的变异
                newSize1 = random.randint(1, sumSize - 1)
                while(newSize1 == self.sublotSizes[pos1] or newSize1 == self.sublotSizes[pos2]):
                    newSize1 = random.randint(1, sumSize - 1)
                self.sublotSizes[pos1] = newSize1
                self.sublotSizes[pos2] = sumSize - newSize1


In [57]:
# 一个个体所有lot的lotSplitingCode
class individualLotSplitingCode:
    
    def __init__(self, lotNum, lotSizes):
        """
        self.lotNum  一个个体有多少个lot
        self.lotSizes  list，每个lot有多少个工件
        """
        self.lotNum = lotNum
        self.lotSizes = lotSizes
        
    def initilizeLotSplitingCode(self):
        """
        根据lotSizes初始化lotNum个lotSplitingVec，随机初始化
        self.lotSplitingCode  list，一个个体所有lot的lotSplitingVec组成的list
        """
        self.lotSplitingCode = []
        for num in self.lotSizes:
            self.lotSplitingCode.append(singleLotSplitingVec(num))
        for item in self.lotSplitingCode:
            item.initializeLotSplitingVec()
            
    def mutateWithinLotWithTwoSublots(self, p):
        """
        按照概率p来随机抽取lot进行lot内两个sublotSize的变异
        p  单个Vec变异的概率
        """
        for item in self.lotSplitingCode:
            if(random.random() < p):
                item.mutateTwoSublot()
                
    def mutateWithinLotWithNewVec(self, p):
        """
        按照概率p来随机抽取lot进行lot内分批方案vec重新随机初始化
        p  单个Vec变异的概率
        """
        for item in self.lotSplitingCode:
            if(random.random() < p):
                item.initializeLotSplitingVec()
                  

In [58]:
# 一个个体所有机器的preferenceCode
class individualPreferenceCode:
    """
    注意：lot号从0开始数
    """
    
    def __init__(self, machineNum, lotNum):
        """
        self.machineNum  机器数量
        self.lotNum  一个个体有多少个lot
        """
        self.machineNum = machineNum
        self.lotNum = lotNum
        
    def initilizePreferenceCode(self):
        """
        随机初始化self.machineNum台机器的preferenceCode
        self.preferenceCode list，一个个体所有机器的preferenceVec组成的list
        """
        self.preferenceCode = []        
        for i in range(self.machineNum):
            preferenceVec = [item for item in range(self.lotNum)]
            random.shuffle(preferenceVec)
            self.preferenceCode.append(preferenceVec)
            
    def mutateWithinVecWithSwap(self, p):
        """
        按照概率p选取机器的preferenceVec进行Vec内两点swap
        p  单个Vec变异的概率
        """
        for item in self.preferenceCode:
            if(random.random() < p):                
                pos1 = random.randint(1, self.lotNum) - 1
                pos2 = random.randint(1, self.lotNum) - 1
                while(pos1 == pos2):
                    pos2 = random.randint(1, self.lotNum) - 1
                item[pos1], item[pos2] = item[pos2], item[pos1]
                
    def mutateBetweenVecsWithSwap(self):
        """
        随机抽取两个preferenceVec，swap
        """
        pos1 = random.randint(1, self.machineNum) - 1
        pos2 = random.randint(1, self.machineNum) - 1
        while(pos1 == pos2):
            pos2 = random.randint(1, self.machineNum) - 1
        self.preferenceCode[pos1], self.preferenceCode[pos2] = self.preferenceCode[pos2], self.preferenceCode[pos1]

In [59]:
# 一个完整的个体
class individual:
    
    def __init__(self, lotNum, lotSizes, machineNum):
        """
        self.lotNum  有多少个lot
        self.lotSizes  list，每个lot有多少个工件
        self.machineNum  有多少个机器
        """
        self.lotNum = lotNum
        self.lotSizes = lotSizes
        self.machineNum = machineNum
        
    def initializeIndividual(self):
        """
        随机初始化一个个体的两段编码
        self.segment1  S1,分批段
        self.segment2  S2,偏好段
        self.makespan  该个体的完工时间，初始值为一个很大的数
        """
        self.segment1 = individualLotSplitingCode(self.lotNum, self.lotSizes)
        self.segment1.initilizeLotSplitingCode()
        self.segment2 = individualPreferenceCode(self.machineNum, self.lotNum)
        self.segment2.initilizePreferenceCode()
        self.makespan = 100000
        
    def mutateSegment1WithTwoSublots(self, p):
        """
        按照概率p随机选择lotSplitingVec，对其随机两个sublot的size扰动
        p  单个Vec变异的概率
        """
        self.segment1.mutateWithinLotWithTwoSublots(p)
        
    def mutateSegment1WithNewVec(self, p):
        """
        按照概率p随机选择lotSplitingVec，对整个向量重新初始化
        p  单个Vec变异的概率
        """
        self.segment1.mutateWithinLotWithNewVec(p)
        
    def mutateSegment2WithinVecWithSwap(self, p):
        """
        按照概率p随机选择preferenceVec，对Vec内两个位置swap
        p  单个Vec变异的概率
        """
        self.segment2.mutateWithinVecWithSwap(p)
        
    def mutateSgment2BetweenTwoVecs(self):
        """
        随机选两个preferenceVec进行swap
        """
        self.segment2.mutateBetweenVecsWithSwap()
        
    def crossoverBetweenSegment1s(indi1, indi2, p):
        """
        按概率p选位，交叉两个individual的segment1的位（以一个lot为一位）
        """
        for i in range(indi1.lotNum):
            if(random.random() < p):
                indi1.segment1.lotSplitingCode[i], indi2.segment1.lotSplitingCode[i] = \
                indi2.segment1.lotSplitingCode[i], indi1.segment1.lotSplitingCode[i]
                
    def crossoverBetweenSegment2s(indi1, indi2, p):
        """
        按概率p选位，交叉两个individual的segment2的位（以一台机器为一位）
        """
        for i in range(indi1.machineNum):
            if(random.random() < p):
                indi1.segment2.preferenceCode[i], indi2.segment2.preferenceCode[i] = \
                indi2.segment2.preferenceCode[i], indi1.segment2.preferenceCode[i]
        
    def decode(self):
        """
        计算并返回完工时间
        """
        solu = solution(self)
        solu.run()
        self.makespan = solu.getMakespan()
#         return self.makespan

        

In [60]:
# 一些外部函数

def calRoulette(makespanList):
    """
    由完工时间列表计算轮盘
    makespanList  输入一个完工时间的list
    roulette      输出一个轮盘概率list
    """
    makespanMax = max(makespanList)
    for i in range(len(makespanList)):
        makespanList[i] = makespanMax - makespanList[i] + 1
    makespanSum = sum(makespanList)    
    for i in range(len(makespanList)):
        makespanList[i] /=  makespanSum
    roulette = []
    temp = 0
    for i in range(len(makespanList)):
        temp += makespanList[i]
        roulette.append(temp)
    return roulette

def chooseOneNumByRoulette(roulette):
    """
    用轮盘来随机选取一个数
    roulette      输入一个轮盘概率list
    i             输出一个[0,len(roulette)-1]的随机数
    """    
    randNum = random.random()
    for i in range(len(roulette)):
        if(randNum < roulette[i]):
            break
    return i

def chooseTwoNumByRoulette(roulette): 
    """
    用轮盘赌来随机选取两个不同的数
    roulette      输入一个轮盘概率list
    pos1, pos2    输出两个不同的[0,len(roulette)-1]的随机数
    """
    pos1 = chooseOneNumByRoulette(roulette)
    pos2 = chooseOneNumByRoulette(roulette)
    while(pos1 == pos2):
        pos2 = chooseOneNumByRoulette(roulette)
    return pos1, pos2

In [61]:
# 全局变量

problemInd = 2

""" 
problemInd               问题编号，需要手动指定
timeMatrix               每种lot的每个工序由不同的机器加工需要多少时间
preparingTimeMatrix      工序准备时间
lotNum                   有多少个lot
machineMatrix            每种lot的每个工序都可以由哪几台机器加工
lotOpeartionNumList      list，每种lot各有多少个工序
machineNum               有多少台机器
operationNumOfMachine    每台机器可以加工多少个不同的工序
"""


# P1：4X6问题，来自王海燕
if(problemInd == 1):
    timeMatrix=[ [ {0: 2, 1: 3, 2: 4}, {1: 3, 3: 2, 4: 4}, {0: 1, 1: 4, 2: 5} ],\
                 [ {0: 3, 2: 5, 4: 2}, {0: 4, 1: 3, 4: 6}, {2: 4, 4: 7, 5: 11} ],\
                 [ {0: 5, 1: 6      }, {1: 4, 3: 3, 4: 5}, {2: 13, 4: 9, 5: 12} ],\
                 [ {0: 9, 2: 7, 3: 9}, {1: 6, 3: 4, 5: 5}, {0: 1, 2: 3, 5: 3} ] ]

    preparingTimeMatrix=[ [ {0: 2, 1: 3, 2: 4}, {1: 3, 3: 2, 4: 4}, {0: 1, 1: 4, 2: 5} ],\
                          [ {0: 3, 2: 5, 4: 2}, {0: 4, 1: 3, 4: 6}, {2: 4, 4: 7, 5: 11} ],\
                          [ {0: 5, 1: 6      }, {1: 4, 3: 3, 4: 5}, {2: 13, 4: 9, 5: 12} ],\
                          [ {0: 9, 2: 7, 3: 9}, {1: 6, 3: 4, 5: 5}, {0: 1, 2: 3, 5: 3} ] ]

    lotSizes = [8,8,8,8]

# P2：4X6问题，来自王海燕
elif(problemInd == 2):
    timeMatrix=[ [ {0: 2, 1: 3, 2: 4}, {1: 3, 3: 2, 4: 4}, {0: 1, 1: 2, 2: 5} ],\
                 [ {0: 3, 2: 5, 4: 2}, {0: 4, 1: 3, 4: 6}, {2: 4, 4: 7, 5: 11} ],\
                 [ {0: 5, 1: 6      }, {1: 4, 3: 3, 4: 5}, {2: 13, 4: 9, 5: 12} ],\
                 [ {0: 9, 2: 7, 3: 9}, {1: 6, 3: 4, 5: 5}, {0: 1, 2: 3, 5: 3} ] ]

    preparingTimeMatrix=[ [ {0: 2, 1: 3, 2: 4}, {1: 3, 3: 2, 4: 4}, {0: 1, 1: 2, 2: 5} ],\
                          [ {0: 3, 2: 5, 4: 2}, {0: 4, 1: 3, 4: 6}, {2: 4, 4: 7, 5: 11} ],\
                          [ {0: 5, 1: 6      }, {1: 4, 3: 3, 4: 5}, {2: 13, 4: 9, 5: 12} ],\
                          [ {0: 9, 2: 7, 3: 9}, {1: 6, 3: 4, 5: 5}, {0: 1, 2: 3, 5: 3} ] ]

    lotSizes = [20,20,20,20]

# P3：6X6问题，来自王海燕
elif(problemInd == 3):
    timeMatrix=[ [ {0: 2}, {2: 3, 3: 2}, {1: 2, 3: 2, 4: 3, 5: 2}, {1: 5, 3: 6}, {2: 2, 5: 2}, {1: 1, 4: 1} ],\
                 [ {1: 2, 3: 1}, {2: 4}, {0: 8, 4: 7, 5: 7}, {1: 4, 2: 5, 3: 5}, {2: 1, 5: 1}, {1: 4, 4: 5} ],\
                 [ {0: 4, 2: 5}, {1: 5, 3: 5}, {2: 1, 4: 1}, {1: 6, 4: 7}, {1: 2, 2: 2, 5: 3} ],\
                 [ {0: 4, 3: 4}, {2: 2}, {1: 4, 3: 3, 5: 3}, {2: 6, 4: 5}, {0: 6}, {1: 3, 3: 2, 4: 2} ],\
                 [ {0: 2, 3: 3}, {1: 5, 4: 4}, {2: 1, 3: 1}, {2: 3, 5: 2}, {1: 3, 2: 2}, {4: 2} ],\
                 [ {0: 2, 2: 3, 4: 2}, {1: 4, 4: 3}, {3: 6, 5: 6}, {1: 2, 3: 2}, {2: 1}, {0: 2, 3: 3, 4: 2} ] ]

    preparingTimeMatrix =[ [ {0: 1}, {2: 2, 3: 1}, {1: 1, 3: 2, 4: 2, 5: 1}, {1: 3, 3: 2}, {2: 1, 5: 1}, {1: 1, 4: 1} ],\
                           [ {1: 1, 3: 1}, {2: 2}, {0: 2, 4: 2, 5: 3}, {1: 2, 2: 1, 3: 2}, {2: 1, 5: 1}, {1: 2, 4: 1} ],\
                           [ {0: 2, 2: 2}, {1: 3, 3: 2}, {2: 1, 4: 1}, {1: 3, 4: 2}, {1: 2, 2: 1, 5: 1} ],\
                           [ {0: 2, 3: 1}, {2: 1}, {1: 1, 3: 1, 5: 1}, {2: 2, 4: 2}, {0: 1}, {1: 1, 3: 2, 4: 1} ],\
                           [ {0: 1, 3: 1}, {1: 1, 4: 1}, {2: 1, 3: 1}, {2: 1, 5: 1}, {1: 1, 2: 2}, {4: 2} ],\
                           [ {0: 1, 2: 1, 4: 2}, {1: 1, 4: 2}, {3: 2, 5: 1}, {1: 1, 3: 1}, {2: 2}, {0: 2, 3: 1, 4: 2} ] ]
    lotSizes = [10,10,10,10,10,10]

# P4：6X6问题，来自王海燕
elif(problemInd == 4):
    timeMatrix=[ [ {0: 2}, {2: 3, 3: 2}, {1: 2, 3: 2, 4: 3, 5: 2}, {1: 5, 3: 6}, {2: 2, 5: 2}, {1: 1, 4: 1} ],\
                 [ {1: 2, 3: 1}, {2: 4}, {0: 8, 4: 7, 5: 7}, {1: 4, 2: 5, 3: 5}, {2: 1, 5: 1}, {1: 4, 4: 5} ],\
                 [ {0: 4, 2: 5}, {1: 5, 3: 5}, {2: 1, 4: 1}, {1: 6, 4: 7}, {1: 2, 2: 2, 5: 3} ],\
                 [ {0: 4, 3: 4}, {2: 2}, {1: 4, 3: 3, 5: 3}, {2: 6, 4: 5}, {0: 6}, {1: 3, 3: 2, 4: 2} ],\
                 [ {0: 2, 3: 3}, {1: 4, 4: 4}, {2: 1, 3: 1}, {2: 3, 5: 2}, {1: 3, 2: 2}, {4: 2} ],\
                 [ {0: 2, 2: 3, 4: 2}, {1: 4, 4: 3}, {3: 6, 5: 6}, {1: 2, 3: 2}, {2: 1}, {0: 2, 3: 3, 4: 2} ] ]

    preparingTimeMatrix =[ [ {0: 1}, {2: 2, 3: 1}, {1: 1, 3: 2, 4: 2, 5: 1}, {1: 3, 3: 2}, {2: 1, 5: 1}, {1: 1, 4: 1} ],\
                           [ {1: 1, 3: 1}, {2: 2}, {0: 2, 4: 2, 5: 3}, {1: 2, 2: 1, 3: 2}, {2: 1, 5: 1}, {1: 2, 4: 1} ],\
                           [ {0: 2, 2: 2}, {1: 3, 3: 2}, {2: 1, 4: 1}, {1: 3, 4: 2}, {1: 2, 2: 1, 5: 1} ],\
                           [ {0: 2, 3: 1}, {2: 1}, {1: 1, 3: 1, 5: 1}, {2: 2, 4: 2}, {0: 1}, {1: 1, 3: 2, 4: 1} ],\
                           [ {0: 1, 3: 1}, {1: 1, 4: 1}, {2: 1, 3: 1}, {2: 1, 5: 1}, {1: 1, 2: 2}, {4: 2} ],\
                           [ {0: 1, 2: 1, 4: 2}, {1: 1, 4: 2}, {3: 2, 5: 1}, {1: 1, 3: 1}, {2: 2}, {0: 2, 3: 1, 4: 2} ] ]
    lotSizes = [20,20,20,20,20,20]
    
# P5：5X12问题，来自ZHAO
elif(problemInd == 5):
    timeMatrix=[ [ {5: 5, 6: 7, 7: 8}, {4: 10}, {3: 2}, {4: 5}, {10: 12, 11: 14} ],\
                 [ {5: 5, 6: 9, 7: 6}, {8: 3, 9: 4}, {1: 4, 2: 4}, {1: 15, 2: 7}, {1: 5, 2: 5}, {10: 10, 11: 8} ],\
                 [ {5: 5, 6: 7, 7: 8}, {5: 6, 6: 6, 7: 10}, {8: 4, 9: 5}, {1: 15, 2: 14}, {1: 5, 2: 3}, {10: 10, 11: 11} ],\
                 [ {4: 6}, {0: 4}, {8: 3, 9: 5}, {4: 5}, {10: 6, 11: 9} ],\
                 [ {5: 5, 6: 6, 7: 6}, {5: 5, 6: 5, 7: 6}, {8: 5, 9: 5}, {3: 6}, {1: 9, 2: 10}, {1: 5, 2: 4}, {10: 9, 11: 10} ] ]

    preparingTimeMatrix =[ [ {5: 5, 6: 5, 7: 5}, {4: 2}, {3: 4}, {4: 2}, {10: 0, 11: 0} ],\
                           [ {5: 5, 6: 4, 7: 7}, {8: 0, 9: 0}, {1: 8, 2: 3}, {1: 4, 2: 7}, {1: 3, 2: 3}, {10: 0, 11: 0} ],\
                           [ {5: 4, 6: 6, 7: 6}, {5: 6, 6: 8, 7: 6}, {8: 0, 9: 0}, {1: 5, 2: 3}, {1: 3, 2: 1}, {10: 0, 11: 0} ],\
                           [ {4: 2}, {0: 2}, {8: 0, 9: 0}, {4: 3}, {10: 0, 11: 0} ],\
                           [ {5: 4, 6: 4, 7: 5}, {5: 6, 6: 6, 7: 6}, {8: 0, 9: 0}, {3: 4}, {1: 2, 2: 4}, {1: 2, 2: 2}, {10: 0, 11: 0} ] ]
    lotSizes = [600,500,1800,2000,500]




# 由上面两个矩阵计算得到一些全局变量
lotNum = len(timeMatrix)

machineMatrix = [ [ [item for item in operation.keys()] for operation in lot ] for lot in timeMatrix ]

lotOpeartionNumList = [len(item) for item in timeMatrix]


            
temp1 = []
temp2 = [] 
operationNumOfMachine = []
for i, item in enumerate(timeMatrix):
    temp1.extend(item * lotSizes[i])
for i, item in enumerate(temp1):
    temp1[i] = list(item.keys())
for item in temp1:
    temp2.extend(item)
    
machineNum = len(set(temp2))    

for i in range(machineNum):
    operationNumOfMachine.append(temp2.count(i))
    

# 打印上述所有参数
print('timeMatrix: ')
for item in timeMatrix:
    print(item)
print('preparingTimeMatrix: ')
for item in preparingTimeMatrix:
    print(item)
print('lotSizes: ', lotSizes)
print('lotNum: ', lotNum)
print('machineMatrix: ')
for item in machineMatrix:
    print(item)
print('lotOpeartionNumList: ', lotOpeartionNumList)
print('machineNum: ', machineNum)
print('operationNumOfMachine: ', operationNumOfMachine)



timeMatrix: 
[{0: 2, 1: 3, 2: 4}, {1: 3, 3: 2, 4: 4}, {0: 1, 1: 2, 2: 5}]
[{0: 3, 2: 5, 4: 2}, {0: 4, 1: 3, 4: 6}, {2: 4, 4: 7, 5: 11}]
[{0: 5, 1: 6}, {1: 4, 3: 3, 4: 5}, {2: 13, 4: 9, 5: 12}]
[{0: 9, 2: 7, 3: 9}, {1: 6, 3: 4, 5: 5}, {0: 1, 2: 3, 5: 3}]
preparingTimeMatrix: 
[{0: 2, 1: 3, 2: 4}, {1: 3, 3: 2, 4: 4}, {0: 1, 1: 2, 2: 5}]
[{0: 3, 2: 5, 4: 2}, {0: 4, 1: 3, 4: 6}, {2: 4, 4: 7, 5: 11}]
[{0: 5, 1: 6}, {1: 4, 3: 3, 4: 5}, {2: 13, 4: 9, 5: 12}]
[{0: 9, 2: 7, 3: 9}, {1: 6, 3: 4, 5: 5}, {0: 1, 2: 3, 5: 3}]
lotSizes:  [20, 20, 20, 20]
lotNum:  4
machineMatrix: 
[[0, 1, 2], [1, 3, 4], [0, 1, 2]]
[[0, 2, 4], [0, 1, 4], [2, 4, 5]]
[[0, 1], [1, 3, 4], [2, 4, 5]]
[[0, 2, 3], [1, 3, 5], [0, 2, 5]]
lotOpeartionNumList:  [3, 3, 3, 3]
machineNum:  6
operationNumOfMachine:  [140, 140, 140, 80, 120, 80]


In [62]:
# 机器类
class machine:
    
    def __init__(self):
        """
        self.idleMoment  从此刻开始，机器空闲了
        self.assignedList  list，已经被安排到该机器的工件，格式为[lot号，sublot号，sublot工件数，工序号]
        self.idlePeriods  该机器的空闲时间段，格式为[起始时间，结束时间]
        self.waitingList  从solution对象属性allMachineWaitingList中挑选出来符合条件的待加工工序
        self.waitingListTime  self.waitingList对应的时间list，里面每一个时间表示，idleMoment后多少时间之后，该sublot的该工序完成，
                            此时间包含等待时间和工序准备时间（如果有的话）
        self.waitingListTimePreparing   每个工序的工序准备时间，如果跟上一个工序lot类型一样的话，为0
        self.chosenOperation  本机器本次所选择的工序
        self.chosenOperationTime  本机器所选择的的工序所需时间
        """
        self.idleMoment = 0
        self.assignedList = []
        self.idlePeriods = []
        
        self.waitingList = []
        self.waitingListTime = []
        self.waitingListTimePreparing = []
        self.chosenOperation = 0
        self.chosenOperationTime = 0        

In [63]:
# 解码算子类，由individual初始化得到
class solution:
    
    def __init__(self, individual):
        """
        self.machineList  由self.machineNum个machine对象构成的list
        self.allMachineWaitingList  待加工的工序构成的list，每个元素格式为[lot号，sublot号，sublotSize，工序号，可以开始加工的时间]
        self.idleMomentList  每台机器的idletime，从self.machineList同步而来，用来方便选择下一台需要安排工件的机器
        self.sublotOperationAssignment  每个sublot的每个工序都由哪个机器加工，起止时间是多少，每个元素格式为[机器号，起始时间，结束时间]
        """
        # 从individual同步而来的信息
        self.lotSplitingCode = [item.sublotSizes for item in individual.segment1.lotSplitingCode]
        self.preferenceCode = individual.segment2.preferenceCode
        self.lotNum = individual.lotNum
        self.machineNum = individual.machineNum
        
        # 下面是主要变量
        self.machineList = [machine() for i in range(self.machineNum)]     
        self.idleMomentList = [item.idleMoment for item in self.machineList]
        self.allMachineWaitingList = []   
        self.sublotOperationAssignment = [[[] for sublot in lot] for lot in self.lotSplitingCode]
        
    def initializeAllMachineWaitingList(self):
        """
        把所有lot的所有sublot的工序0放入self.allMachineWaitingList
        """
        for lotInd, lot in enumerate(self.lotSplitingCode):
            for sublotInd, sublotSize in enumerate(lot):
                self.allMachineWaitingList.append([lotInd, sublotInd, sublotSize, 0, 0])
                
    def chooseTheNextMachine(self):
        """
        self.idleMomentList一定要在每次选机器前重新由self.machineList生成出来，不能偷懒
        chosenIndex  返回值，是选中的机器的index
        后面可以补充：
        ①相同idleMoment的多台机器，加入规则选择好的一个，而不是随便选
        """
        self.idleMomentList = [item.idleMoment for item in self.machineList]
        
        # 选出最先idle的，且能加工工序类别总数最少的机器
        alternativeMachine = []
        for i, item in enumerate(self.idleMomentList):
            if(item == min(self.idleMomentList)):
                alternativeMachine.append(i)
        alternativeMachinePriority = [operationNumOfMachine[i] for i in alternativeMachine]
        chosenIndex = alternativeMachine[alternativeMachinePriority.index(min(alternativeMachinePriority))]

        # 选择最先idle的机器
#         chosenIndex = self.idleMomentList.index(min(self.idleMomentList))
        
        return chosenIndex
    
    def generateWaitingList(self, machineInd, usePreparingTime = 1):
        """
        对一台机器生成waitingList和waitingListTime，即从allMachineWaitingList选出符合条件的工序
        machineInd  是机器的号码，从0开始数
        usePreparingTime  是否使用工序准备时间
        """
        # 先清空该机器的waitingList和waitingListTime
        self.machineList[machineInd].waitingList = []
        self.machineList[machineInd].waitingListTime = []
        self.machineList[machineInd].waitingListTimePreparing = []
        
        # 构建waitingList        
        # 选择能给该机器加工，而且在idleMoment时刻已经能开始加工，或者在idleMoment可以提前工序准备的工件
        if(usePreparingTime == 1):
            for item in self.allMachineWaitingList:
                if(machineInd in machineMatrix[item[0]][item[3]] \
                   and item[4] - self.machineList[machineInd].idleMoment \
                   <= preparingTimeMatrix[item[0]][item[3]][machineInd]):
                    self.machineList[machineInd].waitingList.append(item)
        # 选择能给该机器加工，而且在idleMoment时刻已经能开始加工的工件
        else:
            for item in self.allMachineWaitingList:
                if(machineInd in machineMatrix[item[0]][item[3]] and item[4] <= self.machineList[machineInd].idleMoment):
                    self.machineList[machineInd].waitingList.append(item)      
        
        # 构建和waitingListTime
        for item in self.machineList[machineInd].waitingList:
            tempTime = timeMatrix[item[0]][item[3]][machineInd]*item[2]          
            # 如果算上工序准备时间，考虑提前工序准备，要对tempTime进行如下改造
            if(usePreparingTime == 1):                  
                # 与上一个工序为非同类
                if not (len(self.machineList[machineInd].assignedList) > 0 \
                        and item[0] == self.machineList[machineInd].assignedList[-1][0]): 
                    tempTime += preparingTimeMatrix[item[0]][item[3]][machineInd] 
                    self.machineList[machineInd].waitingListTimePreparing.\
                    append(preparingTimeMatrix[item[0]][item[3]][machineInd])
    
    
                # 与上一个工序为同类
                else:
                    if (self.machineList[machineInd].idleMoment < item[4]):
                        tempTime += (item[4] - self.machineList[machineInd].idleMoment)
                    self.machineList[machineInd].waitingListTimePreparing.append(0)
            self.machineList[machineInd].waitingListTime.append(tempTime)
                    
            
    def chooseAndAssignOperation(self, machineInd, usePreference = 1):
        """
        让一台机器从其waitingList选择一个工序，并插入到时间轴中，并维护self.allMachineWaitingList
        后面可以补充：
        ①用时相同的多个工序，加入规则选择好的一个，而不是选完工时间最小的那个
        ②插入时间轴前，检查空闲时间段能不能插入
        machineInd  是机器的号码，从0开始数
        usePreference  是否使用PreferenceCode来指导工件选择
        """                        
        # 如果使用Preference来选择工件的话，要重新生成waitingList和waitingListTime
        if(usePreference == 1 and len(self.machineList[machineInd].waitingList) != 0):
            # 从该机器的waitingList选择该机器偏好度最靠前的工序集合，放在tempList中
            for ind in self.preferenceCode[machineInd]:
                tempList = []
                tempListTime = []
                waitingListTimePreparing = []
                for i,item in enumerate(self.machineList[machineInd].waitingList):
                    if(item[0] == ind):
                        tempList.append(item)
                        tempListTime.append(self.machineList[machineInd].waitingListTime[i])
                        waitingListTimePreparing.append(self.machineList[machineInd].waitingListTimePreparing[i])
                if(len(tempList) > 0):
                    break
            # 用tempList覆盖掉该机器的waitingList和waitingListTime
            self.machineList[machineInd].waitingList = tempList
            self.machineList[machineInd].waitingListTime = tempListTime
            self.machineList[machineInd].waitingListTimePreparing = waitingListTimePreparing
        
        # 如果waitinglist有元素，选择一个来加工
        if(len(self.machineList[machineInd].waitingList) != 0):                                
            # 选择工序、工序在waitingList里面的序号、sublot该工序的加工时间
            index = self.machineList[machineInd].waitingListTime.index(min(self.machineList[machineInd].waitingListTime))
#             index = [item[2] for item in self.machineList[machineInd].waitingList].index(max([item[2] for item in self.machineList[machineInd].waitingList]))
            operation = self.machineList[machineInd].waitingList[index]
            time = self.machineList[machineInd].waitingListTime[index] 
            timePreparing = self.machineList[machineInd].waitingListTimePreparing[index]
            # 更新相关信息
            self.machineList[machineInd].chosenOperation = operation
            self.machineList[machineInd].chosenOperationTime = time       
#             self.machineList[machineInd].assignedList.append(operation) 
            # (lot号，sublot号，工件数，工序号，最早看额开始时间，实际开始时间，实际结束时间，准备工序时间)
            self.machineList[machineInd].assignedList.\
    append(operation[:] + [self.machineList[machineInd].idleMoment, self.machineList[machineInd].idleMoment + time, timePreparing])  
            # 插入到时间轴里，更新idleMoment
            self.machineList[machineInd].idleMoment += time 
            # 更新各种信息
            self.sublotOperationAssignment[operation[0]][operation[1]].\
        append([machineInd, self.machineList[machineInd].idleMoment - time, self.machineList[machineInd].idleMoment])            
            # 将该工序从self.allMachineWaitingList删除，将该sublot的下一个工序加入self.allMachineWaitingList
            self.allMachineWaitingList.remove(operation)
            if operation[3] != lotOpeartionNumList[operation[0]] - 1:
                self.allMachineWaitingList.append(operation[:3][:]+[operation[3]+1, self.machineList[machineInd].idleMoment])
        # 如果waitinglist没有元素，那么把idleMoment加入到idlePeriods
        else:  
            self.machineList[machineInd].idleMoment += 1
            self.machineList[machineInd].chosenOperation = 0
            self.machineList[machineInd].chosenOperationTime = 0
            if(len(self.machineList[machineInd].idlePeriods) != 0 and self.machineList[machineInd].idlePeriods[-1][-1] == self.machineList[machineInd].idleMoment - 1):  #  如果最新一段空闲时间段跟此刻idleMoment是连续的，那么在那基础上扩展就行
                self.machineList[machineInd].idlePeriods[-1][-1] = self.machineList[machineInd].idleMoment
            else:
                self.machineList[machineInd].idlePeriods.append([self.machineList[machineInd].idleMoment-1, self.machineList[machineInd].idleMoment])

    def run(self, mute = 1):
        """
        自动求解调度方案
        mute  等于1时，不打印求解过程
        """
        #  初始化
        self.initializeAllMachineWaitingList()
        if(mute != 1):
            print('allMachineWaitingList: ', self.allMachineWaitingList)
        
        #  开始循环求解
#         for i in range(80):
        while(len(self.allMachineWaitingList) != 0):
        
            chosenMachine = self.chooseTheNextMachine()
            
            if(mute != 1):
                print('idleMomentList: ', self.idleMomentList)
                print('chosenMachine: ', chosenMachine)
            
            self.generateWaitingList(machineInd = chosenMachine)
            
            if(mute != 1):
                print('waitingList: ', self.machineList[chosenMachine].waitingList)
                print('waitingListTime: ', self.machineList[chosenMachine].waitingListTime)
                print('waitingListTimePreparing: ', self.machineList[chosenMachine].waitingListTimePreparing)
                        
            self.chooseAndAssignOperation(chosenMachine)
            
            if(mute != 1):
                print('waitingList: ', self.machineList[chosenMachine].waitingList)
                print('waitingListTime: ', self.machineList[chosenMachine].waitingListTime)

                print('chosenOperation: ', self.machineList[chosenMachine].chosenOperation)
                print('chosenOperationTime: ', self.machineList[chosenMachine].chosenOperationTime)

                print('assignedList: ', self.machineList[chosenMachine].assignedList)
                print('idleMoment: ', self.machineList[chosenMachine].idleMoment)
                print('idlePeriods: ', self.machineList[chosenMachine].idlePeriods)
                print('allMachineWaitingList: ', self.allMachineWaitingList)
                print('sublotOperationAssignment', self.sublotOperationAssignment)
                print(' ')
                
        # 最后把多余的idlePeriods删掉，才能得到准确的idleMoment和idlePeriods
        for i in range(self.machineNum):
            if(len(self.machineList[i].idlePeriods) != 0):
                if(self.machineList[i].idlePeriods[-1][-1] == self.machineList[i].idleMoment):
                    self.machineList[i].idleMoment = self.machineList[i].idlePeriods[-1][0]
                    del(self.machineList[i].idlePeriods[-1])
            
    def printResults(self):
        """
        打印run()的调度方案信息
        """
        print('assignment for each machine')
        for i,item in enumerate(self.machineList):
            print('for machine %i: '%i, item.assignedList)
        print('idlePeriods for each machine')
        for i,item in enumerate(self.machineList):
            print('for machine %i: '%i, item.idlePeriods)
        print('completion time for each machine: ', [item.idleMoment for item in self.machineList])
        print('total completion time: ', max([item.idleMoment for item in self.machineList]))
        print('sublotOperationAssignment for each sublot:')
        for i,lot in enumerate(solu.sublotOperationAssignment):
            print('for lot%d: '%i)
            print(lot)
            
    
    def getMakespan(self):
        """
        返回完工时间
        """
        return max([item.idleMoment for item in self.machineList])
    
    
    def generateGantTimetable(self, filename = 'gantData.csv'):
        """
        为该solution生成甘特图时间表
        filename  csv文件路径
        """
        gantData = []
        for machInd, machine in enumerate(self.machineList):
            for item in machine.assignedList:
                if(item[7] != 0):
                    gantData.append(['M%d'%machInd, item[5], item[5] + item[7], '*'])
                gantData.append(['M%d'%machInd, item[5] + item[7], item[6], \
                                 '{lotInd}-{sublotInd}-{operationInd}'.format(lotInd = item[0], sublotInd = item[1], operationInd = item[3])])
        df = pd.DataFrame(gantData, columns=["Machine", "Start", "Finish","Title"])
        df.to_csv(PATH+"\\"+filename, header = False)
        print('gantChart timetable', filename, 'done: {}'.format(PATH+"\\"+filename))

            

In [64]:
# 种群类
class population:
    """
    由多个individual组成的种群，具有GA的选择、交叉、变异功能
    
    self.popSize     种群容量
    self.lotNum      有多少个lot
    self.lotSizes    list，每个lot有多少个工件
    self.machineNum  机器数量
    self.details     记录本种群每次迭代DataFrame
    """
    
    def __init__(self, popSize, lotNum, lotSizes, machineNum):
        
        self.popSize = popSize
        self.lotNum = lotNum
        self.lotSizes = lotSizes
        self.machineNum = machineNum
        
        self.pop = [individual(lotNum, lotSizes, machineNum) for i in range(popSize)]
        for item in self.pop:
            item.initializeIndividual()
            
        self.details = pd.DataFrame(columns = ['iter', 'bestMakespan'])
                        
    def calAllMakespan(self):
        """
        对本种群内所有个体解码，计算每个个体的完工时间
        """
        for item in self.pop:
            item.decode()   
            
            
    def getBestMakespan(self):
        """
        功能：       返回整个种群最好的makespan
        """
        return min([item.makespan for item in self.pop])
   
    def iterate(self, iterNum, p1, p2, p3, needcalAllMakespan = 1, muteEveryIter = 0, muteResult = 0, **kw):
        """
        功能：              简单GA迭代，可对同一个population对象连续使用
                            首次使用应把needcalAllMakespan设为1，后面应设为0以减少重复计算
        
        输入：
        iterNum             迭代次数
        p1                  交叉概率
        p2                  segment1变异概率
        p3                  segment2变异概率
        needcalAllMakespan  在循环迭代之前是否需要计算全部个体的makespan，默认为1
        muteEveryIter       如果为0，打印每次迭代种群中最好makespan
        muteResult          如果为0，打印迭代结束后最好makespan
        
        可选输入：
        kw['startIter']     输出的迭代代数从此号码开始，如果不指定就从0开始
        kw['saveDetailsUsingDF']  是否把每一代的最好makespan都记录在一个DataFrame即self.details
        """
        # 第一代在此计算所有individual的makespan
        if needcalAllMakespan == 1:
            self.calAllMakespan()
            
        # 每次执行iterate都要清空这个DF
        self.details = pd.DataFrame(columns = ['iter', 'bestMakespan'])
        
        # 开始迭代
        for iterInd in range(iterNum):
            
            # 计算轮盘list
            roulette = calRoulette([item.makespan for item in self.pop])
                
            # 创建self.pop的副本，下面对副本进行交叉变异的操作
            tempPop = copy.deepcopy(self.pop)
            
            # 随机选择两个个体进行segment1和segment2交叉
            for i in range(int(self.popSize / 2)):
                if(random.random() < p1):  
                    pos1, pos2 = chooseTwoNumByRoulette(roulette)
                    tempPop[pos1].crossoverBetweenSegment1s(tempPop[pos2], 0.5)  # 有50%的位进行交换
                    tempPop[pos1].crossoverBetweenSegment2s(tempPop[pos2], 0.5)
            # 随机选择个体对segment1变异
            for item in tempPop:
                if(random.random() < p2):
                    if(random.random() < 0.5):
                        item.mutateSegment1WithTwoSublots(0.3)
                    else:
                        item.mutateSegment1WithNewVec(0.3)
            # 随机选择个体对segment2变异
            for item in tempPop:
                if(random.random() < p3):
                    if(random.random() < 0.5):
                        item.mutateSegment2WithinVecWithSwap(0.3)
                    else:
                        item.mutateSgment2BetweenTwoVecs()
            
            # 择优保留
            for item in tempPop:
                item.decode()
            for i in range(self.popSize):
                if(tempPop[i].makespan < self.pop[i].makespan):
                    self.pop[i] = tempPop[i]
#                     self.pop[i].makespan = tempPop[i].makespan
            
            # 如果myte为0，才去打印每次迭代最好makespan
            if muteEveryIter == 0:  
                if 'startIter' in kw.keys():
                    print('iter%d:'% (iterInd + kw['startIter']), self.getBestMakespan())
                else:
                    print('iter%d:'% (iterInd ), self.getBestMakespan())
                    
            # 如果saveDetailsUsingDF为1，那么把细节记录到成员变量self.details中
            if 'saveDetailsUsingDF' in kw.keys() and kw['saveDetailsUsingDF'] == 1:
                if 'startIter' in kw.keys():
                    self.details.loc[len(self.details)] = [iterInd + kw['startIter'], self.getBestMakespan()]
                else:
                    self.details.loc[len(self.details)] = [iterInd , self.getBestMakespan()]
                            
        if muteResult == 0:
            print('result after %d iterations:'%iterNum, self.getBestMakespan())
            
       
    

            

In [65]:
# population类测试代码

time_start=time.time()

test=population(200,lotNum,lotSizes,machineNum)  #  (100,4,[8,8,8,8],6)  (100,6,[10,10,10,10,10,10],6)
test.iterate(200, 0.5, 0.5, 0.5, needcalAllMakespan = 1, muteEveryIter = 0, muteResult = 0, startIter = 100, saveDetailsUsingDF = 1)

print('')
time_end=time.time()


print('all makespan:', [item.makespan for item in test.pop])
print('best makespan:', min([item.makespan for item in test.pop]))
print('totally cost',time_end-time_start)
print(test.details)

iter100: 222
iter101: 222
iter102: 222
iter103: 222
iter104: 222
iter105: 222
iter106: 222
iter107: 216
iter108: 216
iter109: 216
iter110: 216
iter111: 216
iter112: 216
iter113: 216
iter114: 216
iter115: 216
iter116: 216
iter117: 207
iter118: 207
iter119: 207
iter120: 207
iter121: 207
iter122: 207
iter123: 207
iter124: 207
iter125: 199
iter126: 199
iter127: 199
iter128: 199
iter129: 199
iter130: 199
iter131: 199
iter132: 199
iter133: 199
iter134: 199
iter135: 199
iter136: 199
iter137: 199
iter138: 199
iter139: 199
iter140: 199
iter141: 199
iter142: 199
iter143: 199
iter144: 199
iter145: 199
iter146: 199
iter147: 199
iter148: 199
iter149: 199
iter150: 199
iter151: 199
iter152: 199
iter153: 199
iter154: 199
iter155: 199
iter156: 199
iter157: 199
iter158: 199
iter159: 199
iter160: 199
iter161: 199
iter162: 199
iter163: 199
iter164: 199
iter165: 194
iter166: 194
iter167: 194
iter168: 194
iter169: 194
iter170: 194
iter171: 194
iter172: 194
iter173: 194
iter174: 194
iter175: 194
iter176: 194

In [66]:
# 将上述种群中最好个体的参数找出来，打印参数，并放入solution类观察其解码过程
"""
P1的最优解，85
[2, 1, 4, 3]
[[4, 4], [8], [2, 2, 2, 2], [3, 3, 2]]
[[2, 1, 0, 3], [1, 2, 0, 3], [3, 1, 0, 2], [2, 3, 0, 1], [2, 3, 0, 1], [2, 3, 0, 1]]

P2的最优解，183
[8, 1, 6, 2]
[[4, 1, 3, 2, 3, 3, 3, 1], [20], [4, 4, 4, 1, 4, 3], [12, 8]]
[[1, 2, 0, 3], [1, 2, 0, 3], [3, 1, 0, 2], [3, 1, 0, 2], [2, 1, 0, 3], [2, 0, 1, 3]]

P3的最优解，201
[4, 5, 2, 6, 4, 5]
[[3, 2, 3, 2], [2, 3, 3, 1, 1], [6, 4], [1, 1, 2, 1, 3, 2], [2, 2, 4, 2], [2, 3, 2, 2, 1]]
[[2, 3, 1, 4, 5, 0], [1, 2, 5, 0, 4, 3], [1, 3, 4, 2, 5, 0], [2, 0, 5, 1, 4, 3], [5, 4, 2, 3, 1, 0], [4, 0, 1, 5, 3, 2]]

P4的最优解，389
[5, 8, 7, 6, 8, 8]
[[1, 7, 5, 5, 2], [3, 2, 2, 2, 3, 4, 1, 3], [4, 1, 3, 2, 2, 3, 5], [3, 4, 5, 3, 1, 4], [3, 2, 1, 4, 1, 4, 2, 3], [5, 2, 2, 1, 5, 3, 1, 1]]
[[2, 0, 4, 1, 3, 5], [0, 2, 1, 3, 4, 5], [1, 3, 5, 0, 2, 4], [0, 2, 1, 4, 5, 3], [4, 5, 3, 2, 1, 0], [1, 3, 0, 4, 5, 2]]

P5的最优解，31227
[3, 2, 28, 85, 43]
[[210, 195, 195], [296, 204], [64, 65, 63, 47, 74, 62, 58, 63, 56, 51, 59, 68, 69, 74, 70, 65, 70, 54, 62, 59, 66, 61, 62, 81, 67, 62, 77, 71], [23, 28, 18, 16, 24, 25, 17, 17, 32, 18, 19, 28, 26, 15, 25, 24, 19, 20, 26, 16, 33, 18, 25, 18, 25, 21, 27, 24, 28, 19, 26, 22, 18, 25, 30, 18, 16, 25, 20, 32, 22, 24, 23, 20, 30, 17, 36, 28, 19, 28, 22, 22, 22, 14, 19, 18, 23, 23, 14, 29, 30, 25, 32, 30, 33, 28, 22, 33, 23, 17, 30, 28, 21, 28, 17, 27, 21, 28, 21, 25, 19, 30, 30, 18, 25], [12, 17, 14, 12, 7, 10, 13, 10, 10, 10, 12, 16, 11, 10, 13, 13, 11, 8, 9, 14, 13, 12, 12, 12, 14, 13, 10, 14, 11, 9, 10, 14, 8, 10, 10, 9, 12, 12, 18, 9, 18, 8, 10]]
[[1, 2, 0, 4, 3], [4, 1, 3, 0, 2], [3, 2, 1, 4, 0], [1, 4, 0, 2, 3], [2, 0, 4, 1, 3], [3, 0, 2, 1, 4], [0, 3, 4, 1, 2], [4, 1, 0, 2, 3], [1, 2, 4, 3, 0], [4, 0, 1, 2, 3], [2, 3, 1, 4, 0], [1, 0, 2, 4, 3]]
"""

# 找最好个体
print('all individual:', [item.makespan for item in test.pop])
print('best makespan:', min([item.makespan for item in test.pop]))

# 打印参数
makespans = [item.makespan for item in test.pop]
bestInd = makespans.index(min(makespans))
print('best index is:', bestInd)

sublotNum = []
sublotSizes = []
preferenceCode = []
for item in test.pop[bestInd].segment1.lotSplitingCode:
    sublotNum.append(item.sublotNum)
for item in test.pop[bestInd].segment1.lotSplitingCode:
    sublotSizes.append(item.sublotSizes)
preferenceCode = test.pop[bestInd].segment2.preferenceCode
    
print('sublotNum')
print(sublotNum)
print('sublotSizes')
print(sublotSizes)
print('preferenceCode')
print(preferenceCode)
print('----------------------------------------------------------------------------------------------------------------------')

# 创建新个体
testIndividual=individual(lotNum,lotSizes,machineNum)
testIndividual.initializeIndividual()

# 深复制最好那个个体
testIndividual=copy.deepcopy(test.pop[bestInd])

# 再次打印参数，核对信息
for item in testIndividual.segment1.lotSplitingCode:
    print(item.lotSize, item.sublotNum, item.sublotSizes)
for item in testIndividual.segment2.preferenceCode:
    print(item)
print('----------------------------------------------------------------------------------------------------------------------')

# 解码，生成甘特图
solu=solution(testIndividual)
solu.run(mute=1)
print('----------------------------------------------------------------------------------------------------------------------')
solu.printResults()
solu.generateGantTimetable()
print('makespan: ', solu.getMakespan())



all individual: [221, 195, 197, 201, 207, 211, 194, 203, 217, 196, 220, 196, 193, 201, 196, 216, 218, 208, 224, 198, 206, 215, 194, 198, 194, 219, 203, 216, 205, 216, 217, 210, 196, 202, 217, 216, 206, 194, 193, 194, 212, 192, 192, 196, 225, 199, 196, 194, 205, 203, 202, 213, 197, 215, 203, 198, 213, 202, 223, 196, 223, 207, 233, 199, 196, 201, 210, 201, 217, 211, 216, 193, 196, 207, 211, 215, 206, 197, 203, 203, 196, 217, 209, 202, 208, 199, 214, 210, 200, 191, 208, 202, 200, 205, 234, 197, 199, 210, 209, 201, 197, 217, 204, 224, 196, 193, 205, 187, 230, 232, 192, 207, 211, 219, 194, 212, 215, 220, 207, 203, 211, 206, 217, 227, 194, 208, 214, 200, 197, 217, 202, 206, 210, 209, 204, 217, 201, 202, 207, 210, 199, 216, 205, 205, 196, 202, 199, 195, 229, 203, 215, 197, 213, 199, 205, 221, 202, 203, 194, 202, 212, 206, 199, 192, 222, 219, 207, 194, 207, 196, 203, 204, 203, 215, 200, 195, 217, 210, 205, 201, 208, 205, 203, 200, 202, 224, 211, 208, 194, 210, 196, 195, 203, 215, 205, 194, 208

In [67]:
# 手动赋值创建新个体，并放入solution类观察其解码过程
[2, 1, 4, 3]
[[4, 4], [8], [2, 2, 2, 2], [3, 3, 2]]
[[2, 1, 0, 3], [1, 2, 0, 3], [3, 1, 0, 2], [2, 3, 0, 1], [2, 3, 0, 1], [2, 3, 0, 1]]
# 创建新个体
testIndividual=individual(lotNum,lotSizes,machineNum)
testIndividual.initializeIndividual()

for i, item in enumerate(testIndividual.segment1.lotSplitingCode):   
    # P1的最优解，85
    item.sublotNum = \
    [2, 1, 4, 3][i]
    item.sublotSizes = \
    [[4, 4], [8], [2, 2, 2, 2], [3, 3, 2]][i]
testIndividual.segment2.preferenceCode = \
    [[2, 1, 0, 3], [1, 2, 0, 3], [3, 1, 0, 2], [2, 3, 0, 1], [2, 3, 0, 1], [2, 3, 0, 1]]

    
# 再次打印参数，核对信息
for item in testIndividual.segment1.lotSplitingCode:
    print(item.lotSize, item.sublotNum, item.sublotSizes)
for item in testIndividual.segment2.preferenceCode:
    print(item)
print('----------------------------------------------------------------------------------------------------------------------')

# 解码，生成甘特图
solu=solution(testIndividual)
solu.run(mute=1)
print('----------------------------------------------------------------------------------------------------------------------')
solu.printResults()
solu.generateGantTimetable()
print('makespan: ', solu.getMakespan())

20 2 [4, 4]
20 1 [8]
20 4 [2, 2, 2, 2]
20 3 [3, 3, 2]
[2, 1, 0, 3]
[1, 2, 0, 3]
[3, 1, 0, 2]
[2, 3, 0, 1]
[2, 3, 0, 1]
[2, 3, 0, 1]
----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
assignment for each machine
for machine 0:  [[2, 0, 2, 0, 0, 0, 15, 5], [2, 2, 2, 0, 0, 15, 25, 0], [2, 3, 2, 0, 0, 25, 35, 0], [0, 0, 4, 0, 0, 35, 45, 2], [0, 1, 4, 0, 0, 45, 53, 0], [3, 2, 2, 2, 54, 53, 56, 1], [0, 0, 4, 2, 60, 59, 64, 1], [3, 0, 3, 2, 66, 65, 69, 1], [0, 1, 4, 2, 72, 71, 76, 1], [3, 1, 3, 2, 78, 77, 81, 1]]
for machine 1:  [[2, 1, 2, 0, 0, 0, 18, 6], [1, 0, 8, 1, 18, 18, 45, 3], [0, 0, 4, 1, 45, 45, 60, 3], [0, 1, 4, 1, 53, 60, 72, 0]]
for machine 2:  [[3, 0, 3, 0, 0, 0, 28, 7], [3, 1, 3, 0, 0, 28, 49, 0], [1, 0, 8, 2, 45, 49, 85, 4]]
for machine 3:  [[3, 2, 2, 0, 0, 0, 27, 9], [2, 1, 2, 1, 18, 27, 36, 

In [68]:
# 一些全局函数

def getBestOrWorstIndexs(mode, makespanList, indNum):
    
    """
    功能：         找出给定makespanList中最好或者最坏的indNum个个体
    
    输入：
    mode           选择模式，可以是'best','worst'
    makespanList   list，给定完工时间列表
    indNum         要选出多少个
    
    输出：
    indexs         一个list，里面是individual在pop中的序号
    """
    
    indexs = []
    temp = copy.deepcopy(makespanList)
    
    if mode == 'best':
        Inf = 99999999
        for i in range(indNum):
            indexs.append(temp.index(min(temp)))
            temp[temp.index(min(temp))]=Inf
    elif mode == 'worst':
        Inf = 0
        for i in range(indNum):
            indexs.append(temp.index(max(temp)))
            temp[temp.index(max(temp))]=Inf
            
    return indexs


In [69]:
class islandModle:
    """
    成员变量：
    self.modelSize       有多少个island（种群）
    self.popSize         每个种群有多少个individual
    self.lotNum          有多少个lot
    self.lotSizes        list，每个lot有多少个工件
    self.machineNum      有多少台机器
    self.model           list，由self.modleSize个population组成
    self.detailsOfModel  记录每一代每一个种群的最好个体
    """
    
    def __init__(self, modelSize, popSize, lotNum, lotSizes, machineNum):
        self.modelSize = modelSize
        self.popSize = popSize
        self.lotNum = lotNum
        self.lotSizes = lotSizes
        self.machineNum = machineNum

        self.model = [population(self.popSize, self.lotNum, self.lotSizes, self.machineNum) for i in range(self.modelSize)]
        
        self.detailsOfModel = pd.DataFrame(columns = ['pop', 'iter', 'outerIter', 'bestMakespan'])

    
    def calAllModelMakespan(self):
        """
        功能：     对所有种群里所有个体计算makespan
        """
        for i in range(self.modelSize):            
            self.model[i].calAllMakespan()
    
    
    def getBestMakespanOfEveryPop(self):
        """
        功能：     返回每个种群最好makespan组成的list
        """
        return [self.model[i].getBestMakespan() for i in range(self.modelSize)]
    
    
    def getBestMakespanAmongAllPops(self):
        """
        功能：     返回所有种群中最好的makespan
        """
        return min(self.getBestMakespanOfEveryPop())
            
        
    def getCertainIndividualOfPopulation(self, popInd, mode, choosePercentage):
        """
        功能：            返回某个种群部分个体在种群中的序号
        
        输入：
        popInd            种群序号
        mode              选择模式，可以是'best','worst',random'
        choosePercentage  选出choosePercentage%个个体，例如可以是10，30等
        
        输出：
        indexs            一个list，包含部分个体在种群中的序号
        """
        
        # 先确定要选多少个个体
        chooseNum = int(choosePercentage * self.popSize / 100.0)
            
        # 找出个体的索引
        if mode == 'best':
            makespanList = [self.model[popInd].pop[i].makespan for i in range(self.popSize)]
            indexs = getBestOrWorstIndexs(mode, makespanList, chooseNum)
        elif mode  == 'worst':
            makespanList = [self.model[popInd].pop[i].makespan for i in range(self.popSize)]
            indexs = getBestOrWorstIndexs(mode, makespanList, chooseNum)
        elif mode == 'random':
            indexs = random.sample(range(self.popSize), chooseNum)
        
        return indexs
            
        
        
    def migrateBetweenTwoPops(self, mode, popIndex1, popIndex2, individualIndexs1, individualIndexs2):
        """
        功能：              在两个种群之间迁移
        
        输入：
        mode                模式，可以是'replace'，或者是'exchange'
        popIndex1           第一个种群序号，如果模式是'replace'，则为源种群序号
        popIndex2           第二个种群序号，如果模式是'replace'，则为目标种群序号
        individualIndexs1   list，第一个种群中需要migrate的个体序号
        individualIndexs2   list，第二个种群中需要migrate的个体序号
        """
        if mode == 'replace':
            for i in range(len(individualIndexs1)):
                self.model[popIndex2].pop[individualIndexs2[i]] = copy.deepcopy(self.model[popIndex1].pop[individualIndexs1[i]])  # 要深copy才保险
        elif mode == 'exchange':
            for i in range(len(individualIndexs1)):
                self.model[popIndex1].pop[individualIndexs1[i]], self.model[popIndex2].pop[individualIndexs2[i]] = \
                self.model[popIndex2].pop[individualIndexs2[i]], self.model[popIndex1].pop[individualIndexs1[i]]
    
    
    
    def migrationOfAllPops(self, mode, choosePercentage):
        """
        功能:              所有种群进行迁移
        
        输入：
        mode               模式，可以是'replace'，或者是'exchange'
        choosePercentage   选出choosePercentage%个个体，例如可以是10，30等
        """
        
        # 先生成每个种群最好和最差的个体index
        migrateIndexs = []
        for i in range(self.modelSize):
            migrateIndexs.append([self.getCertainIndividualOfPopulation(i, mode = 'best', choosePercentage = choosePercentage), \
                                  self.getCertainIndividualOfPopulation(i, mode = 'worst', choosePercentage = choosePercentage)])
        
        # 三个种群按照0-1,1-2,2-0的顺序migration
        for i in range(self.modelSize):
            fromPop = i
            toPop = (i + 1) % self.modelSize
            self.migrateBetweenTwoPops(mode, fromPop, toPop, migrateIndexs[fromPop][0], migrateIndexs[toPop][1])
            
            
    
    def modelIterate(self, outerIterNum, innerIterNum, p1, p2, p3, muteEveryGAIter = 1, muteGAResult = 1, \
                     muteEveryOuterIter = 0, muteOuterResult = 0, **kw):
        
        """
        功能：                      使用简单GA迭代来构建IMGA的迭代
        
        输入：
        outerIterNum                模型要进行多少次migrate
        innerIterNum                每多少个iter就要migrate一次
        p1                          交叉概率
        p2                          segment1变异概率
        p3                          segment2变异概率
        muteEveryGAIter             如果为0，打印每次GA迭代种群中最好makespan
        muteGAResult                如果为0，打印inner迭代结束后最好makespan
        muteEveryOuterIter          如果为0，打印每次outer迭代种群中最好makespan
        muteOuterResult             如果为0，打印outer迭代结束后最好makespan
        
        可选输入：
        kw['saveDetailsUsingDF']   是否生成一个DataFrame来记录详细结果
        """
        
        # 第一次迭代需要手动计算所有个体的makespan
        self.calAllModelMakespan()
        
        #外部迭代
        for outerIterInd in range(outerIterNum):
            
            # 内部迭代
            for popInd in range(self.modelSize):
                # GA
                if 'saveDetailsUsingDF' in kw.keys() and kw['saveDetailsUsingDF'] == 1:
                    saveDetailsUsingDF = kw['saveDetailsUsingDF']
                self.model[popInd].iterate(innerIterNum, p1, p2, p3, needcalAllMakespan = 0, \
                                           muteEveryIter = muteEveryGAIter, muteResult = muteGAResult, \
                                           startIter = outerIterInd * innerIterNum,\
                                          saveDetailsUsingDF = saveDetailsUsingDF)
                # 记录到dataframe里
                if 'saveDetailsUsingDF' in kw.keys() and kw['saveDetailsUsingDF'] == 1:
                    self.model[popInd].details['pop'] = popInd
                    self.model[popInd].details['outerIter'] = outerIterInd
                    self.detailsOfModel = self.detailsOfModel.append(self.model[popInd].details, ignore_index=True)
            
            # 打印完整外部迭代一代后的结果
            if muteEveryOuterIter == 0:
                print('outerIter: %d'%outerIterInd, self.getBestMakespanAmongAllPops(), self.getBestMakespanOfEveryPop())
            
            # 每个外部迭代一代，就迁移一次，迁移的mode在此指定
            self.migrationOfAllPops('exchange', choosePercentage = 30)
            
        if muteOuterResult == 0:
            print('result after {num1} outerIteration and {num2} innerIteration which is {num3} in total:'.\
                  format(num1 = outerIterNum, num2 = innerIterNum, num3 = outerIterNum * innerIterNum), \
                  self.getBestMakespanAmongAllPops())
        
        
                
                
        
        


In [70]:
# islandModle类测试2，与单种群对比

test = islandModle(5, 10, lotNum, lotSizes, machineNum)
test.modelIterate(10, 10, 0.5, 0.5, 0.5, muteEveryGAIter = 1, muteGAResult = 1, muteEveryOuterIter = 1,\
                  muteOuterResult = 0, saveDetailsUsingDF = 1)
print(list(test.detailsOfModel[test.detailsOfModel['bestMakespan'] == min(test.detailsOfModel['bestMakespan'])].head(1)['iter'])[0])

test2=population(50,lotNum,lotSizes,machineNum)  #  (100,4,[8,8,8,8],6)  (100,6,[10,10,10,10,10,10],6)
test2.iterate(100, 0.5, 0.5, 0.5, needcalAllMakespan = 1, muteEveryIter = 1, muteResult = 0, saveDetailsUsingDF = 1)
print(list(test2.details[test2.details['bestMakespan'] == min(test2.details['bestMakespan'])].head(1)['iter'])[0])

result after 10 outerIteration and 10 innerIteration which is 100 in total: 195
99
result after 100 iterations: 203
87


In [73]:
# islandModle类测试3，与单种群对比
islandMakespans = []
GAMakespans = []

islandIterNums = []
GAIterNums = []

In [77]:
# 两种算法对比测试，相同个体数，相同代数
for i in range(20):
    
    print('the %dth time starts'%i)
    
    test1 = islandModle(10, 10, lotNum, lotSizes, machineNum)
    test1.modelIterate(40, 20, 0.5, 0.2, 0.2, muteEveryGAIter = 1, muteGAResult = 1, muteEveryOuterIter = 1,\
                      muteOuterResult = 1, saveDetailsUsingDF = 1)
    islandMakespans.append(test1.getBestMakespanAmongAllPops())
    islandIterNums.append(list(test1.detailsOfModel[test1.detailsOfModel['bestMakespan'] == \
                                                    min(test1.detailsOfModel['bestMakespan'])].head(1)['iter'])[0])
    
    test2=population(100,lotNum,lotSizes,machineNum)  #  (100,4,[8,8,8,8],6)  (100,6,[10,10,10,10,10,10],6)
    test2.iterate(800, 0.5, 0.2, 0.2, needcalAllMakespan = 1, muteEveryIter = 1, muteResult = 1, saveDetailsUsingDF = 1)
    GAMakespans.append(test2.getBestMakespan())
    GAIterNums.append(list(test2.details[test2.details['bestMakespan'] == min(test2.details['bestMakespan'])].head(1)['iter'])[0])
    
print('end!')

the 0th time starts
the 1th time starts
the 2th time starts
the 3th time starts
the 4th time starts
the 5th time starts
the 6th time starts
the 7th time starts
the 8th time starts
the 9th time starts
the 10th time starts
the 11th time starts
the 12th time starts
the 13th time starts
the 14th time starts
the 15th time starts
the 16th time starts
the 17th time starts
the 18th time starts
the 19th time starts
end!


In [78]:
print('makespans')
print(islandMakespans, sum(islandMakespans) / len(islandMakespans))
print(GAMakespans, sum(GAMakespans) / len(GAMakespans))
print('iternums')
print(islandIterNums, sum(islandIterNums) / len(islandIterNums))
print(GAIterNums, sum(GAIterNums) / len(GAIterNums))
print('get 183')
print(islandMakespans.count(183))
print(GAMakespans.count(183))

makespans
[185, 185, 189, 188, 183, 185, 189, 187, 188, 187, 185, 185, 189, 185, 189, 192, 185, 185, 185, 188, 188, 189, 187, 185, 183, 186, 187, 185, 187, 189, 187, 185, 183, 189, 192, 184, 185, 187, 189, 188] 186.725
[185, 185, 189, 189, 183, 187, 185, 187, 186, 185, 185, 187, 189, 189, 185, 183, 187, 187, 187, 184, 189, 187, 189, 185, 187, 185, 185, 187, 185, 185, 185, 185, 185, 189, 189, 185, 185, 183, 188, 189] 186.275
iternums
[742, 702, 75, 570, 636, 269, 231, 760, 501, 369, 423, 410, 84, 377, 769, 447, 470, 475, 448, 523, 366, 199, 394, 268, 480, 452, 339, 477, 618, 156, 285, 531, 322, 164, 450, 615, 669, 236, 118, 489] 422.725
[550, 254, 222, 414, 495, 508, 480, 487, 199, 471, 311, 731, 303, 217, 395, 762, 346, 397, 431, 520, 144, 298, 386, 277, 336, 307, 335, 732, 670, 280, 645, 412, 286, 160, 266, 670, 311, 516, 253, 305] 402.05
get 183
3
3


In [52]:
# 找出最优个体的index

test1.getBestMakespanAmongAllPops()

for i in range(test1.modelSize):
    print([test1.model[i].pop[j].makespan for j in range(test1.popSize)])
print('')

[183, 183, 183, 183, 183, 183, 183, 183, 183, 183]
[246, 183, 183, 186, 183, 183, 183, 183, 183, 183]
[183, 230, 183, 183, 183, 183, 183, 183, 183, 183]
[183, 222, 200, 189, 183, 183, 183, 183, 183, 183]
[183, 230, 183, 183, 222, 183, 183, 183, 183, 183]
[183, 183, 243, 185, 185, 183, 183, 183, 183, 183]
[183, 183, 209, 209, 196, 183, 183, 185, 185, 185]
[183, 183, 183, 240, 208, 185, 188, 185, 188, 185]
[185, 185, 185, 268, 211, 185, 195, 183, 183, 183]
[183, 218, 183, 183, 205, 189, 183, 183, 183, 183]



In [72]:
# 把最优个体的编码打出来

# 创建新个体
testIndividual=individual(lotNum,lotSizes,machineNum)
testIndividual.initializeIndividual()

# 深复制最好那个个体
testIndividual=copy.deepcopy(test1.model[9].pop[2])

sublotNum = []
sublotSizes = []
preferenceCode = []
for item in testIndividual.segment1.lotSplitingCode:
    sublotNum.append(item.sublotNum)
for item in testIndividual.segment1.lotSplitingCode:
    sublotSizes.append(item.sublotSizes)
preferenceCode = testIndividual.segment2.preferenceCode

print('sublotNum')
print(sublotNum)
print('sublotSizes')
print(sublotSizes)
print('preferenceCode')
print(preferenceCode)
print('----------------------------------------------------------------------------------------------------------------------')

# 再次打印参数，核对信息
for item in testIndividual.segment1.lotSplitingCode:
    print(item.lotSize, item.sublotNum, item.sublotSizes)
for item in testIndividual.segment2.preferenceCode:
    print(item)
print('----------------------------------------------------------------------------------------------------------------------')

# 解码，生成甘特图
solu=solution(testIndividual)
solu.run(mute=1)
print('----------------------------------------------------------------------------------------------------------------------')
solu.printResults()
solu.generateGantTimetable()
print('makespan: ', solu.getMakespan())

sublotNum
[8, 1, 6, 2]
sublotSizes
[[4, 1, 3, 2, 3, 3, 3, 1], [20], [4, 4, 4, 1, 4, 3], [12, 8]]
preferenceCode
[[1, 2, 0, 3], [1, 2, 0, 3], [1, 3, 0, 2], [3, 2, 1, 0], [2, 1, 0, 3], [2, 1, 3, 0]]
----------------------------------------------------------------------------------------------------------------------
20 8 [4, 1, 3, 2, 3, 3, 3, 1]
20 1 [20]
20 6 [4, 4, 4, 1, 4, 3]
20 2 [12, 8]
[1, 2, 0, 3]
[1, 2, 0, 3]
[1, 3, 0, 2]
[3, 2, 1, 0]
[2, 1, 0, 3]
[2, 1, 3, 0]
----------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------
assignment for each machine
for machine 0:  [[2, 3, 1, 0, 0, 0, 10, 5], [2, 0, 4, 0, 0, 10, 30, 0], [2, 1, 4, 0, 0, 30, 50, 0], [2, 2, 4, 0, 0, 50, 70, 0], [2, 4, 4, 0, 0, 70, 90, 0], [0, 1, 1, 0, 0, 90, 94, 2], [0, 3, 2, 0, 0, 94, 98, 0], [0, 2, 3, 0, 0, 98, 104, 0], [0, 4, 3, 0, 0, 104, 1

## 简单IMGA和单种群GA效果对比

-----------------------------------------------------------------------------------No.1------------------------------------------------------------------------------------------------

problem 1 40 runs 

IMGA：5 island 10 individuals，20 outeriter，10 inneriter

GA：50 individuals，200iters

makespans
[85, 86, 85, 87, 85, 90, 85, 85, 89, 86, 85, 85, 85, 88, 86, 85, 87, 90, 85, 91, 87, 85, 85, 88, 85, 87, 85, 85, 87, 85, 85, 87, 86, 87, 91, 88, 91, 89, 85, 91] 86.725
[85, 88, 88, 86, 83, 92, 88, 89, 89, 87, 87, 89, 88, 87, 87, 85, 89, 88, 89, 88, 85, 85, 87, 85, 87, 85, 90, 87, 92, 87, 86, 86, 87, 85, 85, 86, 85, 87, 88, 85] 87.05

iternums
[75, 188, 113, 177, 160, 109, 84, 148, 54, 166, 114, 92, 92, 171, 170, 187, 161, 179, 180, 111, 37, 137, 126, 10, 59, 179, 196, 151, 48, 139, 117, 134, 197, 111, 196, 123, 99, 80, 135, 85] 127.25
[95, 101, 198, 36, 144, 97, 154, 107, 184, 92, 52, 190, 37, 123, 199, 188, 173, 110, 86, 96, 140, 179, 112, 179, 196, 164, 182, 118, 93, 131, 182, 162, 99, 21, 116, 163, 96, 64, 180, 185] 130.6

get 85
18
10

结论：分island更容易找到最优解


In [34]:
# islandModle类测试1

print('创建新的IMGA')
test = islandModle(3, 10, lotNum, lotSizes, machineNum)
print('')


print('计算所有种群所有个体的makespan')
print('调用calAllModelMakespan()前')
for i in range(test.modelSize):
    print([item.makespan for item in test.model[i].pop])
test.calAllModelMakespan()
print('调用calAllModelMakespan()后')
for i in range(test.modelSize):
    print([item.makespan for item in test.model[i].pop])
print('')


print('找出每个种群中最好makespan')
print(test.getBestMakespanOfEveryPop())
print('')


print('找出所有种群中最好makespan')
print(test.getBestMakespanAmongAllPops())
print('')

    
print('找出种群0的最好、最差、随机个体')
print(test.getCertainIndividualOfPopulation(0, 'best', choosePercentage = 30))
print(test.getCertainIndividualOfPopulation(0, 'worst', choosePercentage = 30))
print(test.getCertainIndividualOfPopulation(0, 'random', choosePercentage = 30))
print('')


print('用replace模式，把种群0的优秀个体替换掉种群1的最差个体')
print('best of pop0:', test.getCertainIndividualOfPopulation(0, 'best', choosePercentage = 10))
print('best of pop1:', test.getCertainIndividualOfPopulation(1, 'worst', choosePercentage = 10))
test.migrateBetweenTwoPops('replace',0,1,\
                           test.getCertainIndividualOfPopulation(0, 'best', choosePercentage = 10),\
                           test.getCertainIndividualOfPopulation(1, 'worst', choosePercentage = 10))
for i in range(test.modelSize):
    print([test.model[i].pop[j].makespan for j in range(test.popSize)])
print('')

    
print('用exchange模式，把种群0的优秀个体与种群1的最差个体交换')
print('best of pop0:', test.getCertainIndividualOfPopulation(0, 'best', choosePercentage = 10))
print('best of pop1:', test.getCertainIndividualOfPopulation(1, 'worst', choosePercentage = 10))
test.migrateBetweenTwoPops('exchange',0,1,\
                           test.getCertainIndividualOfPopulation(0, 'best', choosePercentage = 10),\
                           test.getCertainIndividualOfPopulation(1, 'worst', choosePercentage = 10))
for i in range(test.modelSize):
    print([test.model[i].pop[j].makespan for j in range(test.popSize)])
print('')

    
print('整个model迁移，replace模式')
test.migrationOfAllPops('replace', choosePercentage = 10)
for i in range(test.modelSize):
    print([test.model[i].pop[j].makespan for j in range(test.popSize)])
print('')

    
print('整个model迁移，exchange模式')
test.migrationOfAllPops('exchange', choosePercentage = 10)
for i in range(test.modelSize):
    print([test.model[i].pop[j].makespan for j in range(test.popSize)])
print('')


test.modelIterate(5, 5, 0.5, 0.5, 0.5)

创建新的IMGA

计算所有种群所有个体的makespan
调用calAllModelMakespan()前
[100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
[100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
[100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]
调用calAllModelMakespan()后
[178, 143, 154, 137, 156, 176, 154, 153, 158, 154]
[153, 119, 118, 137, 150, 168, 127, 181, 133, 130]
[182, 144, 187, 122, 144, 153, 136, 199, 122, 127]

找出每个种群中最好makespan
[137, 118, 122]

找出所有种群中最好makespan
118

找出种群0的最好、最差、随机个体
[3, 1, 7]
[0, 5, 8]
[7, 0, 6]

用replace模式，把种群0的优秀个体替换掉种群1的最差个体
best of pop0: [3]
best of pop1: [7]
[178, 143, 154, 137, 156, 176, 154, 153, 158, 154]
[153, 119, 118, 137, 150, 168, 127, 137, 133, 130]
[182, 144, 187, 122, 144, 153, 136, 199, 122, 127]

用exchange模式，把种群0的优秀个体与种群1的最差个体交换
best of pop0: [3]
best of pop1: [5]
[178, 143, 154, 168, 156, 176, 154, 153, 158, 154]
[153, 119, 118, 137, 150, 137, 127, 137, 133, 130]
[182, 144, 187, 122, 144

UnboundLocalError: local variable 'saveDetailsUsingDF' referenced before assignment